# SCORE DISTRIBUTION


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO

# API URLs
cleaned_data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'

def fetch_csv_from_api(api_url):
    from datetime import datetime, timedelta
    import os
    
    cache_file = 'cleaned_data_cache.csv'
    cache_timeout = timedelta(hours=24)
    
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")
            return pd.read_csv(cache_file)
    
    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False)
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Tải dữ liệu
try:
    data_df = fetch_csv_from_api(cleaned_data_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()


# Load dữ liệu từ file CSV
data_cleaned = pd.read_csv('cleaned_data_cache.csv')

# Hàm tính phân phối điểm
def calculate_score_distribution(data):
    score_columns = ['Toan', 'Van', 'Ly', 'Sinh', 'Ngoai ngu', 'Hoa', 'Lich su', 'Dia ly', 'GDCD']
    all_scores = data[score_columns].values.flatten()
    valid_scores = all_scores[(all_scores >= 0) & (all_scores <= 10)]

    # Tính phân phối điểm
    distribution = pd.cut(valid_scores, bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                           right=False, labels=["0-1", "1-2", "2-3", "3-4", "4-5",
                                                "5-6", "6-7", "7-8", "8-9", "9-10"]).value_counts()
    return distribution

# Lọc dữ liệu theo năm
data_2018 = data_cleaned[data_cleaned['Year'] == 2018]
data_2019 = data_cleaned[data_cleaned['Year'] == 2019]

# Tính phân phối điểm cho từng năm
distribution_2018 = calculate_score_distribution(data_2018)
distribution_2019 = calculate_score_distribution(data_2019)

# Tạo bảng so sánh
score_distribution = pd.DataFrame({
    '2018': distribution_2018,
    '2019': distribution_2019
}).fillna(0).astype(int)

# Hiển thị kết quả
print(score_distribution)


Đang tải dữ liệu từ cache cleaned_data_cache.csv...
Đang tải dữ liệu từ cache tinh_cache.csv...
Đã tải dữ liệu thành công từ API
        2018    2019
0-1     4673    2417
1-2    58697   52028
2-3   418059  359857
3-4   789914  678593
4-5   884148  828060
5-6   896658  950974
6-7   694468  880868
7-8   405080  626975
8-9   173213  305448
9-10   32368   77544
